In [1]:
using JSON
using Random
using Distributions
using MySQL
using DotEnv
using DataFrames
using PrettyTables
cfg = DotEnv.config()
const DB = DBInterface
NUM=5

5

In [2]:
#query = "insert into bootstrap_trials set trial_time=now(),sample_size=?,number_of_tiers=?"
#    create_trial_statement =  DBInterface.prepare(d, query);

In [3]:
 d = DBInterface.connect(MySQL.Connection,cfg["host"], cfg["user"], cfg["password"], db =cfg["database"], port = parse(Int64,cfg["port"]));
query = "select i.name,j.institution_id ,
SUM(CASE
  WHEN j.estimated_tier = 1 THEN count ELSE 0 END
) AS TIER1,
SUM(CASE
  WHEN j.estimated_tier = 2 THEN count ELSE 0 END
) AS TIER2,
SUM(CASE
  WHEN j.estimated_tier = 3 THEN count ELSE 0 END
) AS TIER3,
SUM(CASE
  WHEN j.estimated_tier = 4 THEN count ELSE 0 END
) AS TIER4,
SUM(CASE
  WHEN j.estimated_tier = 5 THEN count ELSE 0 END
) AS TIER5

from ftrecruiter.institutions i join
(select d.institution_id, d.estimated_tier, count(*) as count from bootstrap_data d left join bootstrap_trials t  
on t.trial_id=d.trial_id where t.number_of_tiers=?
group by institution_id, estimated_tier) j on i.institution_id=j.institution_id
group by j.institution_id"
get_institutions =  DBInterface.prepare(d, query);
m = DB.execute(get_institutions, [NUM]) |> DataFrame
println(m)
DB.close!(d)

752×7 DataFrame
 Row │ name                               institution_id  TIER1    TIER2    TIER3    TIER4    TIER5   
     │ String?                            Int32?          Dec64…?  Dec64…?  Dec64…?  Dec64…?  Dec64…? 
─────┼────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ University of California, San Di…               1      2.0     86.0     30.0      0.0      0.0
   2 │ University of California Los Ang…               6    116.0      1.0      1.0      0.0      0.0
   3 │ University of Maryland                         13    116.0      1.0      1.0      0.0      0.0
   4 │ Syracuse University                            15      0.0     15.0    101.0      2.0      0.0
   5 │ Brandeis University                            16      0.0      1.0      9.0     61.0     12.0
   6 │ Duke University                                17    116.0      1.0      1.0      0.0      0.0
   7 │ Northwestern University                        18    116

In [4]:
P = zeros(Int32, (NUM, NUM))
i = 0
for r in eachrow(m)
    if r["TIER1"] >= max(r["TIER2"],r["TIER3"],r["TIER4"],r["TIER5"])
        P[1,1] += r["TIER1"]
        P[1,2] += r["TIER2"]
        P[1,3] += r["TIER3"]
        P[1,4] += r["TIER4"]
        P[1,5] += r["TIER5"]
    elseif r["TIER2"] >= max(r["TIER1"],r["TIER3"],r["TIER4"],r["TIER5"])
        P[2,1] += r["TIER1"]
        P[2,2] += r["TIER2"]
        P[2,3] += r["TIER3"]
        P[2,4] += r["TIER4"]
        P[2,5] += r["TIER5"]
    elseif r["TIER3"] >= max(r["TIER1"],r["TIER2"],r["TIER4"],r["TIER5"])
        P[3,1] += r["TIER1"]
        P[3,2] += r["TIER2"]
        P[3,3] += r["TIER3"]
        P[3,4] += r["TIER4"]
        P[3,5] += r["TIER5"]
    elseif r["TIER4"] >= max(r["TIER1"],r["TIER2"],r["TIER3"],r["TIER5"])
        P[4,1] += r["TIER1"]
        P[4,2] += r["TIER2"]
        P[4,3] += r["TIER3"]
        P[4,4] += r["TIER4"]
        P[4,5] += r["TIER5"]
    elseif r["TIER5"] >= max(r["TIER1"],r["TIER2"],r["TIER3"],r["TIER5"])
        P[5,1] += r["TIER1"]
        P[5,2] += r["TIER2"]
        P[5,3] += r["TIER3"]
        P[5,4] += r["TIER4"]
        P[5,5] += r["TIER5"]
    end
end
P

5×5 Matrix{Int32}:
 1970    11      8     17      0
  225  4641    298     28      0
   19   774  10933   2371    376
    0    75   2646  12467   4488
    0     1     61   6099  23988

In [5]:
N = zeros(Float64,(5,5))
for i in 1:NUM
    for j in 1:NUM
        N[i,j] = P[i,j]/sum(P, dims=2)[i]
    end
end

In [6]:
N

5×5 Matrix{Float64}:
 0.982054    0.00548355  0.00398804  0.00847458  0.0
 0.0433359   0.893875    0.057396    0.00539291  0.0
 0.00131279  0.0534789   0.755407    0.163822    0.0259794
 0.0         0.00381175  0.134479    0.633615    0.228095
 0.0         3.31686e-5  0.00202328  0.202295    0.795648

In [15]:
function nice_table(t_table, NUM)
    headers = []
    names = []
    for i in 1:NUM
        push!(headers, "Tier $i")
        push!(names, "Tier $i")
    end
    pretty_table(t_table, header = headers, row_names=names)
    return pretty_table(t_table, header = headers, row_names=names, backend=Val(:latex))
    
end


nice_table (generic function with 2 methods)

In [16]:
nice_table(N, NUM)

┌────────┬────────────┬────────────┬────────────┬────────────┬───────────┐
│        │     Tier 1 │     Tier 2 │     Tier 3 │     Tier 4 │    Tier 5 │
├────────┼────────────┼────────────┼────────────┼────────────┼───────────┤
│ Tier 1 │   0.982054 │ 0.00548355 │ 0.00398804 │ 0.00847458 │       0.0 │
│ Tier 2 │  0.0433359 │   0.893875 │   0.057396 │ 0.00539291 │       0.0 │
│ Tier 3 │ 0.00131279 │  0.0534789 │   0.755407 │   0.163822 │ 0.0259794 │
│ Tier 4 │        0.0 │ 0.00381175 │   0.134479 │   0.633615 │  0.228095 │
│ Tier 5 │        0.0 │ 3.31686e-5 │ 0.00202328 │   0.202295 │  0.795648 │
└────────┴────────────┴────────────┴────────────┴────────────┴───────────┘
\begin{tabular}{rrrrrr}
  \hline
   & \textbf{Tier 1} & \textbf{Tier 2} & \textbf{Tier 3} & \textbf{Tier 4} & \textbf{Tier 5} \\\hline
  Tier 1 & 0.982054 & 0.00548355 & 0.00398804 & 0.00847458 & 0.0 \\
  Tier 2 & 0.0433359 & 0.893875 & 0.057396 & 0.00539291 & 0.0 \\
  Tier 3 & 0.00131279 & 0.0534789 & 0.755407 & 0.163822 &

In [22]:
using Pkg
Pkg.status("PrettyTables")

      Status `~/code/mapinator/estimation/Project.toml`
  [08abe8d2] PrettyTables v2.2.0


In [23]:
pretty_table(N)

┌────────────┬────────────┬────────────┬────────────┬───────────┐
│     Col. 1 │     Col. 2 │     Col. 3 │     Col. 4 │    Col. 5 │
├────────────┼────────────┼────────────┼────────────┼───────────┤
│   0.982054 │ 0.00548355 │ 0.00398804 │ 0.00847458 │       0.0 │
│  0.0433359 │   0.893875 │   0.057396 │ 0.00539291 │       0.0 │
│ 0.00131279 │  0.0534789 │   0.755407 │   0.163822 │ 0.0259794 │
│        0.0 │ 0.00381175 │   0.134479 │   0.633615 │  0.228095 │
│        0.0 │ 3.31686e-5 │ 0.00202328 │   0.202295 │  0.795648 │
└────────────┴────────────┴────────────┴────────────┴───────────┘
